# Instalar y cargar librerias

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=89ec57322bffb2670e73b01f150f72f2a3a393fbb5dc5e996cf61d3a7c49cba3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Cargar dataset del realtor

In [19]:
df_disney = pd.read_csv('/content/DisneylandReviews.csv', encoding='latin-1')

In [20]:
df_disney.head(10)

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong
5,670591897,3,2019-4,Singapore,"Have been to Disney World, Disneyland Anaheim ...",Disneyland_HongKong
6,670585330,5,2019-4,India,Great place! Your day will go by and you won't...,Disneyland_HongKong
7,670574142,3,2019-3,Malaysia,Think of it as an intro to Disney magic for th...,Disneyland_HongKong
8,670571027,2,2019-4,Australia,"Feel so let down with this place,the Disneylan...",Disneyland_HongKong
9,670570869,5,2019-3,India,I can go on talking about Disneyland. Whatever...,Disneyland_HongKong


# Declarar el modelo sentence-transformer

In [21]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

# Pasa todos los script del dialogo del csv a espacios vectoriales


In [23]:
embeddings = model.encode(df_disney['Review_Text'], batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/667 [00:00<?, ?it/s]

# Guardar los vectores con nueva columna llamada embeddings

In [24]:
df_disney['embeddings'] = embeddings.tolist()

In [25]:
df_disney

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch,embeddings
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong,"[0.018682386726140976, 0.01714111864566803, 0...."
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong,"[-0.02977137081325054, 0.03166365623474121, 0...."
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong,"[0.08182824403047562, 0.01760070025920868, 0.0..."
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong,"[0.07055047899484634, 0.0420130155980587, 0.11..."
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong,"[0.07628581672906876, 0.006095226388424635, 0...."
...,...,...,...,...,...,...,...
42651,1765031,5,missing,United Kingdom,i went to disneyland paris in july 03 and thou...,Disneyland_Paris,"[0.0709405168890953, 0.0025732761714607477, 0...."
42652,1659553,5,missing,Canada,2 adults and 1 child of 11 visited Disneyland ...,Disneyland_Paris,"[0.04321962222456932, 0.0005177444545552135, 0..."
42653,1645894,5,missing,South Africa,My eleven year old daughter and myself went to...,Disneyland_Paris,"[0.05319051817059517, 0.03999072685837746, 0.1..."
42654,1618637,4,missing,United States,"This hotel, part of the Disneyland Paris compl...",Disneyland_Paris,"[0.06958082318305969, -0.0044207144528627396, ..."


# Busqueda semantica vectorial

Ya se puede realuzar la busqueda semantica

In [26]:
query_embedding = model.encode(['Where is the Duck Donal?'])

df_disney['similarity'] = df_disney.embeddings.apply(lambda x : util.cos_sim(x, query_embedding[0] ))

In [28]:
df_disney.sort_values('similarity', ascending=False).head(10)

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch,embeddings,similarity
40636,139820210,5,2012-6,United Kingdom,This is the place to go if you want to feel sp...,Disneyland_Paris,"[0.05792314559221268, 0.02157994918525219, 0.0...",[[tensor(0.3594)]]
4657,347652990,5,2015-3,Qatar,"Best place to visit for the entire family, Lov...",Disneyland_HongKong,"[0.06831009685993195, 0.03200950473546982, 0.0...",[[tensor(0.3513)]]
13001,479722594,5,2017-4,United States,"Amazing as always! The food, the rides, the en...",Disneyland_California,"[0.08720824122428894, 0.05021689459681511, 0.1...",[[tensor(0.3446)]]
25628,143154410,4,2012-10,United States,We had went to California for a couple of reas...,Disneyland_California,"[0.05201033875346184, 0.04411882907152176, 0.0...",[[tensor(0.3434)]]
18633,285178244,5,2014-8,United States,It's the ORIGINAL most MAGICAL PLACE ON EARTH....,Disneyland_California,"[0.040778763592243195, 0.03610407933592796, -0...",[[tensor(0.3416)]]
25894,139770985,4,2012-9,United States,"The rides are great, but it's disheartening wh...",Disneyland_California,"[0.013483249582350254, 0.032445866614580154, 0...",[[tensor(0.3407)]]
19969,251354203,4,2015-1,United States,"Crowded as usual, even for week days in Januar...",Disneyland_California,"[0.07664800435304642, -0.011786368675529957, 0...",[[tensor(0.3363)]]
32419,465309564,2,2017-3,United Kingdom,Just returned from a weekend here. Sorely disa...,Disneyland_Paris,"[0.03429640084505081, 0.013801497407257557, 0....",[[tensor(0.3359)]]
23617,174532026,5,2013-8,New Zealand,"Fantastic place, no litter anywhere. Love it! ...",Disneyland_California,"[0.060279496014118195, -0.009942196309566498, ...",[[tensor(0.3317)]]
5427,314510254,5,2015-1,India,"As the name goes, so the mickey and mini.. The...",Disneyland_HongKong,"[0.07977217435836792, 0.04001065716147423, 0.0...",[[tensor(0.3308)]]
